In [12]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [7]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

zsh:1: command not found: wget


In [7]:
import os
import shutil

new_path = "/Users/rishabhagarwal/Documents/IBM_Data_Engineering/Python_Project"
old_path = "/Users/rishabhagarwal/Downloads/source.zip"
shutil.move(old_path, new_path)

'/Users/rishabhagarwal/Documents/IBM_Data_Engineering/Python_Project/source.zip'

In [8]:
print(os.getcwd())
!unzip source.zip

/Users/rishabhagarwal/Documents/IBM_Data_Engineering/Python_Project
Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


In [9]:
tmpFile = "temp.tmp"
logFile = "logFile.txt"
targetFile = "fullyTransformed.csv"

In [34]:
def extractFromCSV(fileName):
    df = pd.read_csv(fileName)
    return df

def extractFromJSON(fileName):
    print(fileName)
    df = pd.read_json(fileName, lines = True)
    return df

In [42]:
def extractFromXML(fileName):
    df = pd.DataFrame(columns = ["name","height","weight"])
    tree = ET.parse(fileName)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = person.find("height").text
        weight = person.find("weight").text
        
        df.append({"name":name, "height":height,"weight":weight}, ignore_index = True)
    return df

In [78]:
pd.read_csv('source1.csv')

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


In [130]:
def extract():
    extractDF = pd.DataFrame(columns = ["name","height","weight"])
    
    for csvFile in glob.glob("*.csv"):
        extractDF = extractDF.append(extractFromCSV(csvFile), ignore_index = True)
    for jsonFile in glob.glob("*.json"):
        extractDF = extractDF.append(extractFromJSON(jsonFile), ignore_index = True)
    for xmlFile in glob.glob("*.xml"):
        extractDF = extractDF.append(extractFromXML(xmlFile), ignore_index = True)
        
    return extractDF

In [131]:
def transform(df):
    df['height'] = round(df['height']*.0254,2)
    df['weight'] = round(df.weight * 0.45359237,2)
    
    return df

In [132]:
def load(df, targetFile):
    df.to_csv(targetFile)

In [133]:
def log(string):
    timestampFormat = "%Y-%h-%d-%H:%M:%S"
    now = datetime.now()
    timestamp = now.strftime(timestampFormat)
    with open("logfile.txt","a") as f:
        f.write(timestamp+","+string+"\n")

In [134]:
def ETL():
    log("Start of Extraction")
    df = extract()
    log("End of Extraction")
    log("Start of Transform")
    df = transform(df)
    log("End of Transform")
    log("Start of Load")
    load(df,"final.csv")
    log('End of Load')

In [135]:
ETL()

source1.json
source3.json
source2.json


In [141]:
df = extract()
df.head()

source1.json
source3.json
source2.json


,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


In [142]:
df = transform(df)
df.head()

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
